In [12]:
import numpy as np
import pandas as pd
import nltk
import spacy
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from nrclex import NRCLex

In [13]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kbeni\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
# read preprocessed data
data = pd.read_csv("prepro_mergerdata.csv")

C:\Users\kbeni\AppData\Local\Temp\ipykernel_20264\1993008494.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("prepro_mergerdata.csv")


In [14]:
# rename first column to number
data = data.rename(columns={'Unnamed: 0': 'number'})
data

,number,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,...,bidders_country,sentences,cleaned_sentences,stemmed_sentences,lemmatized_sentences,pos_tagged_sentences,dependency_parsed_sentences,sentiment,vader_sentiment,emotion
0,17,2014-07-31,['jaya hind investments pvt ltd'],NaN,NaN,Completed,['6499'],1907056319,2014-08-01: it was announced that on 31/07/14 ...,['2910'],...,['IN'],['2014-08-01: it was announced that on 31/07/1...,['20140801 it was announced that on 310714 jay...,['20140801 it wa announc that on 310714 jaya h...,['20140801 it be announce that on 310714 jaya ...,"[[('20140801', 'CD'), ('it', 'PRP'), ('was', '...","[[('20140801', 'npadvmod', 'announced'), ('it'...",0.9118,positive,"[(positive, 0.25)]"
1,119,2017-07-21,"['guangzhou trauer biotechnology co., ltd']",NaN,2017-07-25,Completed,['2120'],1907184442,2016-11-24: it was announced that Guangzhou Tr...,['2042'],...,['CN'],['2016-11-24: it was announced that Guangzhou ...,['20161124 it was announced that guangzhou tra...,['20161124 it wa announc that guangzhou trauer...,['20161124 it be announce that guangzhou traue...,"[[('20161124', 'CD'), ('it', 'PRP'), ('was', '...","[[('20161124', 'npadvmod', 'announced'), ('it'...",0.6908,positive,"[(positive, 0.5454545454545454)]"
2,143,NaN,['alconix corporation'],2020-12-03,NaN,Completed Assumed,['4672'],1941467148,2020-11-25: it was announced that Alconix Corp...,['2521'],...,['JP'],['2020-11-25: it was announced that Alconix Co...,['20201125 it was announced that alconix corpo...,['20201125 it wa announc that alconix corpor h...,['20201125 it be announce that alconix corpora...,"[[('20201125', 'CD'), ('it', 'PRP'), ('was', '...","[[('20201125', 'npadvmod', 'announced'), ('it'...",0.0258,neutral,"[(positive, 0.38461538461538464)]"
3,161,NaN,"['tianjin winspirit energy technology co., ltd']",2019-06-01,NaN,Completed Assumed,['7219'],1907220901,2017-06-01: it was announced that Tianjin Wins...,['7219'],...,['CN'],['2017-06-01: it was announced that Tianjin Wi...,['20170601 it was announced that tianjin winsp...,['20170601 it wa announc that tianjin winspiri...,['20170601 it be announce that tianjin winspir...,"[[('20170601', 'CD'), ('it', 'PRP'), ('was', '...","[[('20170601', 'npadvmod', 'announced'), ('it'...",0.6369,positive,"[(positive, 0.4)]"
4,163,NaN,"['tenneco automotive (thailand) co., ltd']",2013-06-30,NaN,Completed Assumed,['2932'],1633043238,2011-06-30: it was announced that Yarnapund PC...,['2932'],...,['TH'],['2011-06-30: it was announced that Yarnapund ...,['20110630 it was announced that yarnapund pcl...,['20110630 it wa announc that yarnapund pcl ha...,['20110630 it be announce that yarnapund pcl h...,"[[('20110630', 'CD'), ('it', 'PRP'), ('was', '...","[[('20110630', 'npadvmod', 'announced'), ('it'...",0.9260,positive,"[(positive, 0.3)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36130,825710,NaN,['vestum ab'],2021-10-05,NaN,Completed Assumed,['4399'],1941588290,2021-09-15: it was announced that Vestum AB wi...,"['4299', '4299']",...,['SE'],['2021-09-15: it was announced that Vestum AB ...,['20210915 it was announced that vestum ab wil...,['20210915 it wa announc that vestum ab will a...,['20210915 it be announce that vestum ab will ...,"[[('20210915', 'CD'), ('it', 'PRP'), ('was', '...","[[('20210915', 'npadvmod', 'announced'), ('it'...",0.8271,positive,"[(trust, 0.2727272727272727), (positive, 0.272..."
36131,825721,NaN,"['shanghai zj bio-tech co., ltd']",2019-02-08,NaN,Completed Assumed,['2120'],1907197166,2017-02-08: it was announced that Shanghai ZJ ...,['7219'],...,['CN'],['2017-02-08: it was announced that Shanghai Z...,['20170208 it was announced that shanghai zj b...,['20170208 it wa announc that shanghai zj biot...,['20170208 it be announce that shanghai zj bio...,"[[('20170208', 'CD'), ('it', 'PRP'), ('was', '...","[[('20170208', 'npadvmod', 'announced'), ('it'...",0.

In [15]:
# feature engineering using TF-IDF
lemmatized_text = data['lemmatized_sentences'].apply(lambda x: " ".join(eval(x)))

tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(lemmatized_text)

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

tfidf_df

,10,100,1000,10000,100000,11,12,125,13,14,...,zar,zavod,zhang,zhao,zhejiang,zhengzhou,zhongyou,zhou,zhuhai,zoo
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.097359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36130,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36131,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36132,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36133,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# feature engineering using BERT embeddings

model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each lemmatized sentence
sentence_embeddings = model.encode(lemmatized_text.tolist())

# Convert to DataFrame
embeddings_df = pd.DataFrame(sentence_embeddings)
print(embeddings_df.head())

        0         1         2         3         4         5         6    \
0 -0.052809 -0.038020 -0.016260 -0.046810 -0.011117  0.087446  0.002804   
1 -0.044205 -0.048216 -0.027712 -0.042624 -0.028832 -0.032182  0.044947   
2 -0.104274 -0.022494 -0.051749 -0.012982 -0.030465 -0.032546  0.078908   
3 -0.041102  0.023040  0.019482  0.013597 -0.028759 -0.007089  0.076990   
4 -0.085538  0.024729  0.008048  0.005930 -0.046559  0.020254  0.039339   

        7         8         9    ...       374       375       376       377  \
0  0.106155  0.009057  0.017922  ... -0.042708  0.045368 -0.141863  0.019185   
1  0.047600 -0.052291 -0.011209  ... -0.035693  0.077422 -0.095244  0.032258   
2  0.046898  0.011335 -0.026496  ... -0.023023  0.029806 -0.054259 -0.011677   
3  0.011950 -0.050941 -0.024979  ...  0.001755  0.069395 -0.067680  0.055994   
4  0.015351  0.011217 -0.011510  ... -0.051002  0.072906 -0.006913 -0.006133   

        378       379       380       381       382       383  
0 -0

In [32]:
# save the model
joblib.dump(model, 'bert_model.pkl')

# save the embeddings
embeddings_df.to_csv("embeddings.csv")

['bert_model.pkl']

In [7]:
# # sentiment analysis
# sia = SentimentIntensityAnalyzer()
#
# data['sentiment'] = lemmatized_text.apply(lambda x: sia.polarity_scores(x)['compound'])
# print(data[['lemmatized_sentences', 'sentiment']])

                                    lemmatized_sentences  sentiment
0      ['20140801 it be announce that on 310714 jaya ...     0.9118
1      ['20161124 it be announce that guangzhou traue...     0.6908
2      ['20201125 it be announce that alconix corpora...     0.0258
3      ['20170601 it be announce that tianjin winspir...     0.6369
4      ['20110630 it be announce that yarnapund pcl h...     0.9260
...                                                  ...        ...
36130  ['20210915 it be announce that vestum ab will ...     0.8271
36131  ['20170208 it be announce that shanghai zj bio...     0.2263
36132  ['20190909 it be announce that dimet siam pcl ...     0.8126
36133  ['20171219 it be announce that xinjiang tianfu...     0.5994
36134  ['20230928 it be announce that central plain e...    -0.6124

[36135 rows x 2 columns]


In [16]:
# vader sentiment analysis
sia = SentimentIntensityAnalyzer()

data['sentiment'] = data['lemmatized_sentences'].apply(lambda x: sia.polarity_scores(x)['compound'])

def classify_sentiment(score):
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

data['vader_sentiment'] = data['sentiment'].apply(classify_sentiment)

data

,number,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,...,bidders_country,sentences,cleaned_sentences,stemmed_sentences,lemmatized_sentences,pos_tagged_sentences,dependency_parsed_sentences,sentiment,vader_sentiment,emotion
0,17,2014-07-31,['jaya hind investments pvt ltd'],NaN,NaN,Completed,['6499'],1907056319,2014-08-01: it was announced that on 31/07/14 ...,['2910'],...,['IN'],['2014-08-01: it was announced that on 31/07/1...,['20140801 it was announced that on 310714 jay...,['20140801 it wa announc that on 310714 jaya h...,['20140801 it be announce that on 310714 jaya ...,"[[('20140801', 'CD'), ('it', 'PRP'), ('was', '...","[[('20140801', 'npadvmod', 'announced'), ('it'...",0.9118,positive,"[(positive, 0.25)]"
1,119,2017-07-21,"['guangzhou trauer biotechnology co., ltd']",NaN,2017-07-25,Completed,['2120'],1907184442,2016-11-24: it was announced that Guangzhou Tr...,['2042'],...,['CN'],['2016-11-24: it was announced that Guangzhou ...,['20161124 it was announced that guangzhou tra...,['20161124 it wa announc that guangzhou trauer...,['20161124 it be announce that guangzhou traue...,"[[('20161124', 'CD'), ('it', 'PRP'), ('was', '...","[[('20161124', 'npadvmod', 'announced'), ('it'...",0.6908,positive,"[(positive, 0.5454545454545454)]"
2,143,NaN,['alconix corporation'],2020-12-03,NaN,Completed Assumed,['4672'],1941467148,2020-11-25: it was announced that Alconix Corp...,['2521'],...,['JP'],['2020-11-25: it was announced that Alconix Co...,['20201125 it was announced that alconix corpo...,['20201125 it wa announc that alconix corpor h...,['20201125 it be announce that alconix corpora...,"[[('20201125', 'CD'), ('it', 'PRP'), ('was', '...","[[('20201125', 'npadvmod', 'announced'), ('it'...",0.0258,neutral,"[(positive, 0.38461538461538464)]"
3,161,NaN,"['tianjin winspirit energy technology co., ltd']",2019-06-01,NaN,Completed Assumed,['7219'],1907220901,2017-06-01: it was announced that Tianjin Wins...,['7219'],...,['CN'],['2017-06-01: it was announced that Tianjin Wi...,['20170601 it was announced that tianjin winsp...,['20170601 it wa announc that tianjin winspiri...,['20170601 it be announce that tianjin winspir...,"[[('20170601', 'CD'), ('it', 'PRP'), ('was', '...","[[('20170601', 'npadvmod', 'announced'), ('it'...",0.6369,positive,"[(positive, 0.4)]"
4,163,NaN,"['tenneco automotive (thailand) co., ltd']",2013-06-30,NaN,Completed Assumed,['2932'],1633043238,2011-06-30: it was announced that Yarnapund PC...,['2932'],...,['TH'],['2011-06-30: it was announced that Yarnapund ...,['20110630 it was announced that yarnapund pcl...,['20110630 it wa announc that yarnapund pcl ha...,['20110630 it be announce that yarnapund pcl h...,"[[('20110630', 'CD'), ('it', 'PRP'), ('was', '...","[[('20110630', 'npadvmod', 'announced'), ('it'...",0.8885,positive,"[(positive, 0.3)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36130,825710,NaN,['vestum ab'],2021-10-05,NaN,Completed Assumed,['4399'],1941588290,2021-09-15: it was announced that Vestum AB wi...,"['4299', '4299']",...,['SE'],['2021-09-15: it was announced that Vestum AB ...,['20210915 it was announced that vestum ab wil...,['20210915 it wa announc that vestum ab will a...,['20210915 it be announce that vestum ab will ...,"[[('20210915', 'CD'), ('it', 'PRP'), ('was', '...","[[('20210915', 'npadvmod', 'announced'), ('it'...",0.7096,positive,"[(trust, 0.2727272727272727), (positive, 0.272..."
36131,825721,NaN,"['shanghai zj bio-tech co., ltd']",2019-02-08,NaN,Completed Assumed,['2120'],1907197166,2017-02-08: it was announced that Shanghai ZJ ...,['7219'],...,['CN'],['2017-02-08: it was announced that Shanghai Z...,['20170208 it was announced that shanghai zj b...,['20170208 it wa announc that shanghai zj biot...,['20170208 it be announce that shanghai zj bio...,"[[('20170208', 'CD'), ('it', 'PRP'), ('was', '...","[[('20170208', 'npadvmod', 'announced'), ('it'...",0.

In [20]:
# Named entity recognition
nlp = spacy.load('en_core_web_sm')

def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

data["named_entities"] = lemmatized_text.apply(extract_entities)
print(data[['lemmatized_sentences', 'named_entities']])

                                    lemmatized_sentences  \
0      ['20140801 it be announce that on 310714 jaya ...   
1      ['20161124 it be announce that guangzhou traue...   
2      ['20201125 it be announce that alconix corpora...   
3      ['20170601 it be announce that tianjin winspir...   
4      ['20110630 it be announce that yarnapund pcl h...   
...                                                  ...   
36130  ['20210915 it be announce that vestum ab will ...   
36131  ['20170208 it be announce that shanghai zj bio...   
36132  ['20190909 it be announce that dimet siam pcl ...   
36133  ['20171219 it be announce that xinjiang tianfu...   
36134  ['20230928 it be announce that central plain e...   

                                          named_entities  
0      [(20140801, DATE), (310714, CARDINAL), (pvt lt...  
1      [(20161124, DATE), (guangzhou, GPE), (25 milli...  
2      [(20201125, DATE), (184000, CARDINAL), (92 per...  
3      [(20170601, DATE), (tianjin, GPE), (

In [21]:
data

,number,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,...,targets_name,bidders_country,sentences,cleaned_sentences,stemmed_sentences,lemmatized_sentences,pos_tagged_sentences,dependency_parsed_sentences,sentiment,named_entities
0,17,2014-07-31,['jaya hind investments pvt ltd'],NaN,NaN,Completed,['6499'],1907056319,2014-08-01: it was announced that on 31/07/14 ...,['2910'],...,['force motors ltd'],['IN'],['2014-08-01: it was announced that on 31/07/1...,['20140801 it was announced that on 310714 jay...,['20140801 it wa announc that on 310714 jaya h...,['20140801 it be announce that on 310714 jaya ...,"[[('20140801', 'CD'), ('it', 'PRP'), ('was', '...","[[('20140801', 'npadvmod', 'announced'), ('it'...",0.9118,"[(20140801, DATE), (310714, CARDINAL), (pvt lt..."
1,119,2017-07-21,"['guangzhou trauer biotechnology co., ltd']",NaN,2017-07-25,Completed,['2120'],1907184442,2016-11-24: it was announced that Guangzhou Tr...,['2042'],...,"['guangzhou chuangermei biotechnology co., ltd']",['CN'],['2016-11-24: it was announced that Guangzhou ...,['20161124 it was announced that guangzhou tra...,['20161124 it wa announc that guangzhou trauer...,['20161124 it be announce that guangzhou traue...,"[[('20161124', 'CD'), ('it', 'PRP'), ('was', '...","[[('20161124', 'npadvmod', 'announced'), ('it'...",0.6908,"[(20161124, DATE), (guangzhou, GPE), (25 milli..."
2,143,NaN,['alconix corporation'],2020-12-03,NaN,Completed Assumed,['4672'],1941467148,2020-11-25: it was announced that Alconix Corp...,['2521'],...,"['fujine sangyo co., ltd']",['JP'],['2020-11-25: it was announced that Alconix Co...,['20201125 it was announced that alconix corpo...,['20201125 it wa announc that alconix corpor h...,['20201125 it be announce that alconix corpora...,"[[('20201125', 'CD'), ('it', 'PRP'), ('was', '...","[[('20201125', 'npadvmod', 'announced'), ('it'...",0.0258,"[(20201125, DATE), (184000, CARDINAL), (92 per..."
3,161,NaN,"['tianjin winspirit energy technology co., ltd']",2019-06-01,NaN,Completed Assumed,['7219'],1907220901,2017-06-01: it was announced that Tianjin Wins...,['7219'],...,"['guizhou weiyuan technology co., ltd']",['CN'],['2017-06-01: it was announced that Tianjin Wi...,['20170601 it was announced that tianjin winsp...,['20170601 it wa announc that tianjin winspiri...,['20170601 it be announce that tianjin winspir...,"[[('20170601', 'CD'), ('it', 'PRP'), ('was', '...","[[('20170601', 'npadvmod', 'announced'), ('it'...",0.6369,"[(20170601, DATE), (tianjin, GPE), (guizhou we..."
4,163,NaN,"['tenneco automotive (thailand) co., ltd']",2013-06-30,NaN,Completed Assumed,['2932'],1633043238,2011-06-30: it was announced that Yarnapund PC...,['2932'],...,"['walker exhaust (thailand) co., ltd']",['TH'],['2011-06-30: it was announced that Yarnapund ...,['20110630 it was announced that yarnapund pcl...,['20110630 it wa announc that yarnapund pcl ha...,['20110630 it be announce that yarnapund pcl h...,"[[('20110630', 'CD'), ('it', 'PRP'), ('was', '...","[[('20110630', 'npadvmod', 'announced'), ('it'...",0.9260,"[(20110630, DATE), (290611, CARDINAL), (automo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36130,825710,NaN,['vestum ab'],2021-10-05,NaN,Completed Assumed,['4399'],1941588290,2021-09-15: it was announced that Vestum AB wi...,"['4299', '4299']",...,"['hanell entreprenad i gavle ab', 'per lennart...",['SE'],['2021-09-15: it was announced that Vestum AB ...,['20210915 it was announced that vestum ab wil...,['20210915 it wa announc that vestum ab will a...,['20210915 it be announce that vestum ab will ...,"[[('20210915', 'CD'), ('it', 'PRP'), ('was', '...","[[('20210915', 'npadvmod', 'announced'), ('it'...",0.8271,"[(20210915, DATE), (326890, CARDINAL), (99, CA..."
36131,825721,NaN,"['shanghai zj bio-tech co., ltd']",2019-02-08,NaN,Completed Assumed,['2120'],1907197166,2017-02-08: it was announced that Shanghai ZJ ...,['7219'],..

In [22]:
# save the data
data.to_csv("feature_engineered.csv")

In [24]:
# topic modelling
count_vectorizer = CountVectorizer(max_features=1000, stop_words='english')
doc_term_matrix = count_vectorizer.fit_transform(lemmatized_text)

lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(doc_term_matrix)

for i, topic in enumerate(lda.components_):
    print(f"Top 10 words for topic #{i}:")
    print([count_vectorizer.get_feature_names_out()[index] for index in topic.argsort()[-10:]])
    print("\n")

Top 10 words for topic #0:
['deal', 'sa', 'financial', 'provider', 'service', 'transaction', 'company', 'disclose', 'acquire', 'announce']


Top 10 words for topic #1:
['usd', 'approval', 'shareholder', 'capital', 'issue', 'consideration', 'announce', 'transaction', 'million', 'share']


Top 10 words for topic #2:
['represent', 'value', 'base', 'report', 'announce', 'stake', 'price', 'offer', 'cent', 'share']


Top 10 words for topic #3:
['consideration', 'completion', 'transaction', 'chinabase', 'announce', 'million', 'technology', 'stake', 'cny', 'cent']


Top 10 words for topic #4:
['100', 'gas', 'disclose', 'company', 'service', 'mr', 'announce', 'acquire', 'cent', 'stake']




In [27]:
# topics value
topics_values = lda.transform(doc_term_matrix)
data['topic'] = topics_values.argmax(axis=1)
data

,number,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,...,bidders_country,sentences,cleaned_sentences,stemmed_sentences,lemmatized_sentences,pos_tagged_sentences,dependency_parsed_sentences,sentiment,named_entities,topic
0,17,2014-07-31,['jaya hind investments pvt ltd'],NaN,NaN,Completed,['6499'],1907056319,2014-08-01: it was announced that on 31/07/14 ...,['2910'],...,['IN'],['2014-08-01: it was announced that on 31/07/1...,['20140801 it was announced that on 310714 jay...,['20140801 it wa announc that on 310714 jaya h...,['20140801 it be announce that on 310714 jaya ...,"[[('20140801', 'CD'), ('it', 'PRP'), ('was', '...","[[('20140801', 'npadvmod', 'announced'), ('it'...",0.9118,"[(20140801, DATE), (310714, CARDINAL), (pvt lt...",2
1,119,2017-07-21,"['guangzhou trauer biotechnology co., ltd']",NaN,2017-07-25,Completed,['2120'],1907184442,2016-11-24: it was announced that Guangzhou Tr...,['2042'],...,['CN'],['2016-11-24: it was announced that Guangzhou ...,['20161124 it was announced that guangzhou tra...,['20161124 it wa announc that guangzhou trauer...,['20161124 it be announce that guangzhou traue...,"[[('20161124', 'CD'), ('it', 'PRP'), ('was', '...","[[('20161124', 'npadvmod', 'announced'), ('it'...",0.6908,"[(20161124, DATE), (guangzhou, GPE), (25 milli...",3
2,143,NaN,['alconix corporation'],2020-12-03,NaN,Completed Assumed,['4672'],1941467148,2020-11-25: it was announced that Alconix Corp...,['2521'],...,['JP'],['2020-11-25: it was announced that Alconix Co...,['20201125 it was announced that alconix corpo...,['20201125 it wa announc that alconix corpor h...,['20201125 it be announce that alconix corpora...,"[[('20201125', 'CD'), ('it', 'PRP'), ('was', '...","[[('20201125', 'npadvmod', 'announced'), ('it'...",0.0258,"[(20201125, DATE), (184000, CARDINAL), (92 per...",4
3,161,NaN,"['tianjin winspirit energy technology co., ltd']",2019-06-01,NaN,Completed Assumed,['7219'],1907220901,2017-06-01: it was announced that Tianjin Wins...,['7219'],...,['CN'],['2017-06-01: it was announced that Tianjin Wi...,['20170601 it was announced that tianjin winsp...,['20170601 it wa announc that tianjin winspiri...,['20170601 it be announce that tianjin winspir...,"[[('20170601', 'CD'), ('it', 'PRP'), ('was', '...","[[('20170601', 'npadvmod', 'announced'), ('it'...",0.6369,"[(20170601, DATE), (tianjin, GPE), (guizhou we...",3
4,163,NaN,"['tenneco automotive (thailand) co., ltd']",2013-06-30,NaN,Completed Assumed,['2932'],1633043238,2011-06-30: it was announced that Yarnapund PC...,['2932'],...,['TH'],['2011-06-30: it was announced that Yarnapund ...,['20110630 it was announced that yarnapund pcl...,['20110630 it wa announc that yarnapund pcl ha...,['20110630 it be announce that yarnapund pcl h...,"[[('20110630', 'CD'), ('it', 'PRP'), ('was', '...","[[('20110630', 'npadvmod', 'announced'), ('it'...",0.9260,"[(20110630, DATE), (290611, CARDINAL), (automo...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36130,825710,NaN,['vestum ab'],2021-10-05,NaN,Completed Assumed,['4399'],1941588290,2021-09-15: it was announced that Vestum AB wi...,"['4299', '4299']",...,['SE'],['2021-09-15: it was announced that Vestum AB ...,['20210915 it was announced that vestum ab wil...,['20210915 it wa announc that vestum ab will a...,['20210915 it be announce that vestum ab will ...,"[[('20210915', 'CD'), ('it', 'PRP'), ('was', '...","[[('20210915', 'npadvmod', 'announced'), ('it'...",0.8271,"[(20210915, DATE), (326890, CARDINAL), (99, CA...",0
36131,825721,NaN,"['shanghai zj bio-tech co., ltd']",2019-02-08,NaN,Completed Assumed,['2120'],1907197166,2017-02-08: it was announced that Shanghai ZJ ...,['7219'],...,['CN'],['2017-02-08: it was announced that Shanghai Z...,['20170208 it was announced that shanghai zj b...,['20170208 it wa announc that shanghai zj biot...,['20170208 it be announce that shanghai zj bio...,"[[('20170208', 'CD'),

In [30]:
# save the data
data.to_csv("topic_modelled.csv")

In [31]:
# save the model
import joblib
joblib.dump(lda, 'lda_model.pkl')

['lda_model.pkl']

In [8]:
# vader sentiment analysis

sia = SentimentIntensityAnalyzer()

data['vader_sentiment'] = data['lemmatized_sentences'].apply(lambda x: sia.polarity_scores(x)['compound'])

def classify_sentiment(score):
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

data['vader_sentiment'] = data['vader_sentiment'].apply(classify_sentiment)

,number,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,...,targets_name,bidders_country,sentences,cleaned_sentences,stemmed_sentences,lemmatized_sentences,pos_tagged_sentences,dependency_parsed_sentences,sentiment,vader_sentiment
0,17,2014-07-31,['jaya hind investments pvt ltd'],NaN,NaN,Completed,['6499'],1907056319,2014-08-01: it was announced that on 31/07/14 ...,['2910'],...,['force motors ltd'],['IN'],['2014-08-01: it was announced that on 31/07/1...,['20140801 it was announced that on 310714 jay...,['20140801 it wa announc that on 310714 jaya h...,['20140801 it be announce that on 310714 jaya ...,"[[('20140801', 'CD'), ('it', 'PRP'), ('was', '...","[[('20140801', 'npadvmod', 'announced'), ('it'...",0.9118,positive
1,119,2017-07-21,"['guangzhou trauer biotechnology co., ltd']",NaN,2017-07-25,Completed,['2120'],1907184442,2016-11-24: it was announced that Guangzhou Tr...,['2042'],...,"['guangzhou chuangermei biotechnology co., ltd']",['CN'],['2016-11-24: it was announced that Guangzhou ...,['20161124 it was announced that guangzhou tra...,['20161124 it wa announc that guangzhou trauer...,['20161124 it be announce that guangzhou traue...,"[[('20161124', 'CD'), ('it', 'PRP'), ('was', '...","[[('20161124', 'npadvmod', 'announced'), ('it'...",0.6908,positive
2,143,NaN,['alconix corporation'],2020-12-03,NaN,Completed Assumed,['4672'],1941467148,2020-11-25: it was announced that Alconix Corp...,['2521'],...,"['fujine sangyo co., ltd']",['JP'],['2020-11-25: it was announced that Alconix Co...,['20201125 it was announced that alconix corpo...,['20201125 it wa announc that alconix corpor h...,['20201125 it be announce that alconix corpora...,"[[('20201125', 'CD'), ('it', 'PRP'), ('was', '...","[[('20201125', 'npadvmod', 'announced'), ('it'...",0.0258,neutral
3,161,NaN,"['tianjin winspirit energy technology co., ltd']",2019-06-01,NaN,Completed Assumed,['7219'],1907220901,2017-06-01: it was announced that Tianjin Wins...,['7219'],...,"['guizhou weiyuan technology co., ltd']",['CN'],['2017-06-01: it was announced that Tianjin Wi...,['20170601 it was announced that tianjin winsp...,['20170601 it wa announc that tianjin winspiri...,['20170601 it be announce that tianjin winspir...,"[[('20170601', 'CD'), ('it', 'PRP'), ('was', '...","[[('20170601', 'npadvmod', 'announced'), ('it'...",0.6369,positive
4,163,NaN,"['tenneco automotive (thailand) co., ltd']",2013-06-30,NaN,Completed Assumed,['2932'],1633043238,2011-06-30: it was announced that Yarnapund PC...,['2932'],...,"['walker exhaust (thailand) co., ltd']",['TH'],['2011-06-30: it was announced that Yarnapund ...,['20110630 it was announced that yarnapund pcl...,['20110630 it wa announc that yarnapund pcl ha...,['20110630 it be announce that yarnapund pcl h...,"[[('20110630', 'CD'), ('it', 'PRP'), ('was', '...","[[('20110630', 'npadvmod', 'announced'), ('it'...",0.9260,positive


In [9]:
data

,number,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,...,targets_name,bidders_country,sentences,cleaned_sentences,stemmed_sentences,lemmatized_sentences,pos_tagged_sentences,dependency_parsed_sentences,sentiment,vader_sentiment
0,17,2014-07-31,['jaya hind investments pvt ltd'],NaN,NaN,Completed,['6499'],1907056319,2014-08-01: it was announced that on 31/07/14 ...,['2910'],...,['force motors ltd'],['IN'],['2014-08-01: it was announced that on 31/07/1...,['20140801 it was announced that on 310714 jay...,['20140801 it wa announc that on 310714 jaya h...,['20140801 it be announce that on 310714 jaya ...,"[[('20140801', 'CD'), ('it', 'PRP'), ('was', '...","[[('20140801', 'npadvmod', 'announced'), ('it'...",0.9118,positive
1,119,2017-07-21,"['guangzhou trauer biotechnology co., ltd']",NaN,2017-07-25,Completed,['2120'],1907184442,2016-11-24: it was announced that Guangzhou Tr...,['2042'],...,"['guangzhou chuangermei biotechnology co., ltd']",['CN'],['2016-11-24: it was announced that Guangzhou ...,['20161124 it was announced that guangzhou tra...,['20161124 it wa announc that guangzhou trauer...,['20161124 it be announce that guangzhou traue...,"[[('20161124', 'CD'), ('it', 'PRP'), ('was', '...","[[('20161124', 'npadvmod', 'announced'), ('it'...",0.6908,positive
2,143,NaN,['alconix corporation'],2020-12-03,NaN,Completed Assumed,['4672'],1941467148,2020-11-25: it was announced that Alconix Corp...,['2521'],...,"['fujine sangyo co., ltd']",['JP'],['2020-11-25: it was announced that Alconix Co...,['20201125 it was announced that alconix corpo...,['20201125 it wa announc that alconix corpor h...,['20201125 it be announce that alconix corpora...,"[[('20201125', 'CD'), ('it', 'PRP'), ('was', '...","[[('20201125', 'npadvmod', 'announced'), ('it'...",0.0258,neutral
3,161,NaN,"['tianjin winspirit energy technology co., ltd']",2019-06-01,NaN,Completed Assumed,['7219'],1907220901,2017-06-01: it was announced that Tianjin Wins...,['7219'],...,"['guizhou weiyuan technology co., ltd']",['CN'],['2017-06-01: it was announced that Tianjin Wi...,['20170601 it was announced that tianjin winsp...,['20170601 it wa announc that tianjin winspiri...,['20170601 it be announce that tianjin winspir...,"[[('20170601', 'CD'), ('it', 'PRP'), ('was', '...","[[('20170601', 'npadvmod', 'announced'), ('it'...",0.6369,positive
4,163,NaN,"['tenneco automotive (thailand) co., ltd']",2013-06-30,NaN,Completed Assumed,['2932'],1633043238,2011-06-30: it was announced that Yarnapund PC...,['2932'],...,"['walker exhaust (thailand) co., ltd']",['TH'],['2011-06-30: it was announced that Yarnapund ...,['20110630 it was announced that yarnapund pcl...,['20110630 it wa announc that yarnapund pcl ha...,['20110630 it be announce that yarnapund pcl h...,"[[('20110630', 'CD'), ('it', 'PRP'), ('was', '...","[[('20110630', 'npadvmod', 'announced'), ('it'...",0.9260,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36130,825710,NaN,['vestum ab'],2021-10-05,NaN,Completed Assumed,['4399'],1941588290,2021-09-15: it was announced that Vestum AB wi...,"['4299', '4299']",...,"['hanell entreprenad i gavle ab', 'per lennart...",['SE'],['2021-09-15: it was announced that Vestum AB ...,['20210915 it was announced that vestum ab wil...,['20210915 it wa announc that vestum ab will a...,['20210915 it be announce that vestum ab will ...,"[[('20210915', 'CD'), ('it', 'PRP'), ('was', '...","[[('20210915', 'npadvmod', 'announced'), ('it'...",0.8271,positive
36131,825721,NaN,"['shanghai zj bio-tech co., ltd']",2019-02-08,NaN,Completed Assumed,['2120'],1907197166,2017-02-08: it was announced that Shanghai ZJ ...,['7219'],...,"['shanghai zj biomedical technology co., ltd']",['CN'],['2017-02-08: it was announced that Shanghai Z...,['20170208 it was announced that shanghai zj b...,['20170208 it wa announc that shanghai zj biot...,['20170208 it be announce that shanghai zj bio...,

In [11]:
# emotion detection
data['emotion'] = data['lemmatized_sentences'].apply(lambda x: NRCLex(" ".join(eval(x))).top_emotions)

# Display results
print(data[['lemmatized_sentences', 'emotion']].head())

                                lemmatized_sentences  \
0  ['20140801 it be announce that on 310714 jaya ...   
1  ['20161124 it be announce that guangzhou traue...   
2  ['20201125 it be announce that alconix corpora...   
3  ['20170601 it be announce that tianjin winspir...   
4  ['20110630 it be announce that yarnapund pcl h...   

                             emotion  
0                 [(positive, 0.25)]  
1   [(positive, 0.5454545454545454)]  
2  [(positive, 0.38461538461538464)]  
3                  [(positive, 0.4)]  
4                  [(positive, 0.3)]  
